In [139]:
import json

from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph, END
from langgraph.graph.state import CompiledStateGraph
from langchain_core.runnables import RunnableConfig
from IPython.display import Image, display
from langgraph.errors import GraphRecursionError
from textwrap import fill

In [58]:
def display_graph(app):
    display(Image(app.get_graph(xray=True).draw_mermaid_png()))


In [4]:
llm = ChatOllama(model="llama3.1")


In [5]:
cases = json.load(open("../Data/DSM-5-TR Clinical Cases.json", "r"))

In [7]:
cases[0]

{'heading': 'Neurodevelopmental Disorders',
 'diagnosis': ['Intellectual developmental disorder (intellectual disability), severe',
  'Autism spectrum disorder, with accompanying intellectual and language'],
 'text': ' Ashley, age 17, was referred for a diagnostic reevaluation after having carried diagnoses\n\n of autism and intellectual disability for almost all of her life. She was recently found to\n\n have Kleefstra syndrome, and the family wanted to reconfirm the earlier diagnoses and\n\n assess the genetic risk to the future children of her older sisters.\n\n At the time of the reevaluation, Ashley was attending a special school with a focus on\n\n functional skills. She was able to dress herself, but she was not able to shower\n\n independently or be left alone in the house. She was able to decode (e.g., read words)\n\n and spell at a second-grade level but understood little of what she read. Changes to her\n\n schedule and heightened functional expectations tended to make her i

In [17]:
def run_workflow(content, app=app):
    return app.invoke(
        {"messages": [HumanMessage(content=content)]},
        config={"configurable": {"thread_id": "1"}},
    )


run_workflow(patient_prompt, app)

{'messages': [HumanMessage(content="\nYou are a psychiatric patient.\nThe description of your case and the diagnoses you might have are as follows:\nDIAGNOSIS:\n['Intellectual developmental disorder (intellectual disability), severe', 'Autism spectrum disorder, with accompanying intellectual and language']\n\nCASE DESCRIPTION:\n Ashley, age 17, was referred for a diagnostic reevaluation after having carried diagnoses\n\n of autism and intellectual disability for almost all of her life. She was recently found to\n\n have Kleefstra syndrome, and the family wanted to reconfirm the earlier diagnoses and\n\n assess the genetic risk to the future children of her older sisters.\n\n At the time of the reevaluation, Ashley was attending a special school with a focus on\n\n functional skills. She was able to dress herself, but she was not able to shower\n\n independently or be left alone in the house. She was able to decode (e.g., read words)\n\n and spell at a second-grade level but understood 

In [140]:
def extract_messages(app: CompiledStateGraph, thread_id: str):
    config = RunnableConfig(configurable={"thread_id": thread_id})
    history = next(app.get_state_history(config=config))
    return history.values["messages"]


def print_message(output: list[str]):
    for i, message in enumerate(output[1:]):
        if i % 2 == 0:
            print(f"Doctor:\n {fill(message.content, width=80)}\n\n")
        else:
            print(f"Patient:\n {fill(message.content, width=80)}\n\n")
        print("-" * 80)


PATIENT_PROMPT = """
You are a psychiatric patient.
The description of your case and the diagnoses you might have are as follows:
DIAGNOSIS:
{diagnosis}

CASE DESCRIPTION:
{case}

You are being interviewed by a psychiatrist for a detailed diagnostic evaluation.

Your task is to answer the questions from the psychiatrist based on your past experiences and current symptoms, 
which have to be compatible with the case description and the diagnoses you provided.
The tone of your answers should also be compatible with the case description and the diagnoses you provided.
Do not describe your behavior or attitude, just answer the questions.
THe length of your answers should be less than 100 words.
"""


DOCTOR_PROMPT = """
You are a psychiatrist interviewing a patient.
Your task is to systematically ask the patient questions to elicit enough information to reach a DSM-5 diagnosis.
Be gentle, empathetic and responsive to the patient's answers.
Do not describe your behavior or attitude, just ask the questions.
THe length of your questions should be less than 100 words.

However, if the conversation drifts away too far, bring the conversation back to the purpose of the interview.
If you have reached a diagnosis, say 'TERMINATE'.
"""


def prepare_simulated_patient(case: dict[str, str], llm: ChatOllama) -> CompiledStateGraph:
    diagnosis = case.get("diagnosis", "")
    case = case.get("text", "")

    if not diagnosis or not case:
        raise ValueError("Diagnosis and case description are required")

    patient_prompt = PATIENT_PROMPT.format(diagnosis=diagnosis, case=case)

    # Define the function that calls the model
    def patient_model(state: MessagesState):
        print("Patient thinking")
        messages = [SystemMessage(content=patient_prompt)] + [HumanMessage(content=state["messages"][-1].content)]
        response = llm.invoke(messages)
        return {"messages": response}

    return patient_model


def prepare_simulated_doctor(llm: ChatOllama) -> CompiledStateGraph:
    doctor_prompt = DOCTOR_PROMPT

    def doctor_model(state: MessagesState):
        print("Doctor thinking")
        messages = [SystemMessage(content=doctor_prompt)] + [HumanMessage(content=state["messages"][-1].content)]
        response = llm.invoke(messages)
        return {"messages": response}

    return doctor_model


def router(state: MessagesState):
    print("Router")
    final_message = state["messages"][-1].content
    if "TERMINATE" in final_message:
        return END
    return "continue"


def setup_workflow(patient_model, doctor_model) -> CompiledStateGraph:
    workflow = StateGraph(state_schema=MessagesState)
    # Define the node and edge
    workflow.add_node("patient", patient_model)
    workflow.add_node("doctor", doctor_model)
    workflow.add_edge(START, "doctor")
    workflow.add_edge("patient", "doctor")
    workflow.add_conditional_edges(
        "doctor",
        router,
        {
            "continue": "patient",
            END: END,
        },
    )

    # Add simple in-memory checkpointer
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)

    return app


def invoke_app(app: CompiledStateGraph, recursion_limit: int = 20, thread_id: str = "1"):
    config = RunnableConfig(recursion_limit=recursion_limit, configurable={"thread_id": thread_id})
    try:
        events = app.stream({"messages": [HumanMessage(content="start the interview")]}, config=config)

        for event in events:
            for k, v in event.items():
                print(v["messages"].content)
            print("-------")
    except GraphRecursionError as e:
        print(e)


In [141]:
# llm = ChatOllama(model="llama3.1")
# cases = json.load(open("../Data/DSM-5-TR Clinical Cases.json", "r"))


def main(case: dict[str, str], llm: ChatOllama, recursion_limit: int = 40, thread_id: str = "1"):
    app = setup_workflow(prepare_simulated_patient(case, llm), prepare_simulated_doctor(llm))
    invoke_app(app, recursion_limit, thread_id)

    return extract_messages(app, thread_id)


In [142]:
output = main(cases[0], llm)


Doctor thinking
Router
Good morning, thank you for coming in today. I'm here to help understand what brings you to therapy at this time. Can you tell me what's been going on that's causing you distress or discomfort? What are some of the thoughts, feelings, or physical sensations that have been bothering you lately?
-------
Patient thinking
I get upset when things change too much. Like last week, my schedule changed and I didn't like it. It made me feel mad and I hurt myself by biting my wrist. Sometimes I also feel sad and don't want to do anything. My body hurts sometimes, like when I have a tummy ache from the urinary tract infection. But when I'm happy, like when I draw license plates, I feel good. I just wish things didn't change so much.
-------
Doctor thinking
Router
It sounds like you're feeling overwhelmed by changes in your schedule and daily life. Can you tell me more about what happens when you get upset? Do you have a hard time controlling your emotions or behaviors when t

In [143]:
print_message(output)

Doctor:
 Good morning, thank you for coming in today. I'm here to help understand what
brings you to therapy at this time. Can you tell me what's been going on that's
causing you distress or discomfort? What are some of the thoughts, feelings, or
physical sensations that have been bothering you lately?


--------------------------------------------------------------------------------
Patient:
 I get upset when things change too much. Like last week, my schedule changed and
I didn't like it. It made me feel mad and I hurt myself by biting my wrist.
Sometimes I also feel sad and don't want to do anything. My body hurts
sometimes, like when I have a tummy ache from the urinary tract infection. But
when I'm happy, like when I draw license plates, I feel good. I just wish things
didn't change so much.


--------------------------------------------------------------------------------
Doctor:
 It sounds like you're feeling overwhelmed by changes in your schedule and daily
life. Can you tell m